# 加速：多線程爬蟲



* 了解知乎 API 使用方式與回傳內容
* 撰寫程式存取 API 且添加標頭

## 作業目標

* 找一個之前實作過的爬蟲改用多線程改寫，比較前後時間的差異。





In [1]:
import time
import requests
import random
from bs4 import BeautifulSoup
import pandas as pd
import re
import _thread

C:\ProgramData\Anaconda3\lib\site-packages\requests\__init__.py:80: RequestsDependencyWarning: urllib3 (1.25.9) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [3]:
websites = 'https://free-proxy-list.net/'
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36'}

proxy_ips = []

resp = requests.get(websites, headers = headers)
html = BeautifulSoup(resp.text,  "lxml")

x = html.find('tbody').find_all('tr')
ip = ''
ip_last = ''
ip_final = ''

for tr in x[:10]:
    ip = tr.find_all('td')[0].text
    ip_last = tr.find_all('td')[1].text
    ip_final = ip + ':' + ip_last
    proxy_ips.append(ip_final)

In [4]:
proxy_ips

['161.202.226.194:80',
 '78.42.42.41:8080',
 '1.226.23.121:80',
 '134.209.29.120:8080',
 '5.252.161.48:8080',
 '191.101.39.27:80',
 '37.49.127.234:8080',
 '89.221.223.204:80',
 '103.86.155.102:55443',
 '132.145.18.53:80']

In [7]:
# 原始程式碼
startTime = time.time()

for url in proxy_ips:
    print('Use', url)
    try:
        resp = requests.get('http://ip.filefab.com/index.php',
                        proxies={'http': url, 'https': url}, timeout=5)
        soup = BeautifulSoup(resp.text, 'html5lib')
        print(soup.find('h1', id='ipd').text.strip())
    except:
        print('Fail')
        
finishTime = time.time()
print('原程式碼所需時間：', finishTime - startTime)

Use 161.202.226.194:80
Fail
Use 78.42.42.41:8080
Fail
Use 1.226.23.121:80
Fail
Use 134.209.29.120:8080
Your IP address: 134.209.29.120
Use 5.252.161.48:8080
Your IP address: 5.252.161.48
Use 191.101.39.27:80
Your IP address: 191.101.39.27
Use 37.49.127.234:8080
Fail
Use 89.221.223.204:80
Your IP address: 89.221.223.234
Use 103.86.155.102:55443
Fail
Use 132.145.18.53:80
Fail
原程式碼所需時間： 37.79351830482483


In [13]:
# 多工處理程式碼
def get_url(url):
    print('Use', url)
    try:
        resp = requests.get('http://ip.filefab.com/index.php',
                        proxies={'http': url, 'https': url}, timeout=5)
        soup = BeautifulSoup(resp.text, 'html5lib')
        print(soup.find('h1', id = 'ipd').text.strip())
    except:
        print('Fail')
    

startTime = time.time()

for url in proxy_ips:
    _thread.start_new_thread( get_url, (url,) )
    
finishTime = time.time()


Use 161.202.226.194:80
Use 37.49.127.234:8080
Use 191.101.39.27:80
Use 132.145.18.53:80
UseUse 78.42.42.41:8080
Use 134.209.29.120:8080
UseUseUse 5.252.161.48:8080 103.86.155.102:55443
  1.226.23.121:80
89.221.223.204:80

Fail
Fail
Your IP address: 5.252.161.48
Your IP address: 191.101.39.27
Your IP address: 134.209.29.120
Fail
Your IP address: 89.221.223.234
Fail
Fail
Fail


In [14]:
print('多工處理所需時間：', finishTime - startTime)

多工處理所需時間： 0.001998424530029297
